1-imports e path

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

ROOT = Path.cwd().parents[1]
DATA_DIR = ROOT / "data"

INPUT_FILE  = DATA_DIR / "costs.csv"
OUTPUT_FILE = DATA_DIR / "costs_cleaned.csv"

print("Raiz:", ROOT)
print("Entrada:", INPUT_FILE)
print("Saída:", OUTPUT_FILE)


Raiz: /Users/augusto/Library/Mobile Documents/com~apple~CloudDocs/git/avalensurance-bia
Entrada: /Users/augusto/Library/Mobile Documents/com~apple~CloudDocs/git/avalensurance-bia/data/costs.csv
Saída: /Users/augusto/Library/Mobile Documents/com~apple~CloudDocs/git/avalensurance-bia/data/costs_cleaned.csv


2-lendo

In [2]:
df = pd.read_csv(INPUT_FILE)
print("Dimensões originais:", df.shape)
display(df.head())


Dimensões originais: (64800, 11)


,cost_id,person_id,policy_id,record_id,visit_id,annual_medical_cost,annual_premium,monthly_premium,claims_count,avg_claim_amount,total_claims_paid
0,cost_id_49197,64470,policy_id_00037,record_id_15635,visit_id_13715,5845.93,921.51,76.79,1.0,3917.22,3917.22
1,cost_id_15209,13405,policy_id_00069,record_id_18609,visit_id_30336,9024.53,1292.94,107.74,NaN,NaN,0.00
2,cost_id_37383,26180,policy_id_00069,record_id_28693,visit_id_53039,NaN,348.85,NaN,NaN,439.60,439.60
3,cost_id_48770,37875,policy_id_00123,record_id_54086,visit_id_47919,5340.85,732.72,61.06,2.0,1921.08,3842.16
4,cost_id_61529,5498,policy_id_00003,record_id_11096,visit_id_56767,4189.75,612.22,51.02,2.0,1856.81,3713.62


3-Preenchendo vazios

In [3]:
cols_target = ["annual_medical_cost","annual_premium","monthly_premium","claims_count","avg_claim_amount","total_claims_paid"]
df[cols_target] = df[cols_target].replace("", np.nan)
df[cols_target] = df[cols_target].fillna("-1")
display(df[cols_target].isna().sum())


annual_medical_cost    0
annual_premium         0
monthly_premium        0
claims_count           0
avg_claim_amount       0
total_claims_paid      0
dtype: int64

4- Ordenação + x duplicadas

In [4]:
df = df.sort_values(by="person_id", ascending=True)
dup_count = df.duplicated(subset="person_id").sum()
print(f"person_id duplicados antes de remover: {dup_count}")
df = df.drop_duplicates(subset="person_id", keep="first").reset_index(drop=True)
print("Dimensões após deduplicação:", df.shape)
display(df.head())


person_id duplicados antes de remover: 0
Dimensões após deduplicação: (64800, 11)


,cost_id,person_id,policy_id,record_id,visit_id,annual_medical_cost,annual_premium,monthly_premium,claims_count,avg_claim_amount,total_claims_paid
0,cost_id_31150,1,policy_id_00033,record_id_36092,visit_id_40611,839.85,290.63,24.22,1.0,410.76,410.76
1,cost_id_61371,2,policy_id_00119,record_id_45363,visit_id_48818,709.72,278.13,23.18,1.0,-1,269.0
2,cost_id_63662,3,policy_id_00176,record_id_64734,visit_id_01246,1589.27,372.57,31.05,4.0,270.0,-1
3,cost_id_17436,4,policy_id_00112,record_id_43770,visit_id_47786,3246.81,677.54,56.46,0.0,0.0,0.0
4,cost_id_59155,6,policy_id_00017,record_id_61337,visit_id_35905,1462.55,350.4,29.2,3.0,219.58,658.74


5- organização

In [5]:
import numpy as np
import pandas as pd

x_col = "claims_count"
y_col = "avg_claim_amount"
z_col = "total_claims_paid"

tmp = df[[x_col, y_col, z_col]].apply(pd.to_numeric, errors="coerce")

mask_x = df[x_col].astype(str).eq("-1") | tmp[x_col].eq(-1)
mask_y = df[y_col].astype(str).eq("-1") | tmp[y_col].eq(-1)
mask_z = df[z_col].astype(str).eq("-1") | tmp[z_col].eq(-1)


two_zero_mask_x = (tmp[y_col].eq(0) & tmp[z_col].eq(0))
two_zero_mask_y = (tmp[x_col].eq(0) & tmp[z_col].eq(0))
two_zero_mask_z = (tmp[x_col].eq(0) & tmp[y_col].eq(0))

df.loc[two_zero_mask_x & mask_x, x_col] = 0
df.loc[two_zero_mask_y & mask_y, y_col] = 0
df.loc[two_zero_mask_z & mask_z, z_col] = 0


tmp = df[[x_col, y_col, z_col]].apply(pd.to_numeric, errors="coerce")
mask_x = df[x_col].astype(str).eq("-1") | tmp[x_col].eq(-1)
mask_y = df[y_col].astype(str).eq("-1") | tmp[y_col].eq(-1)
mask_z = df[z_col].astype(str).eq("-1") | tmp[z_col].eq(-1)


x_fix = mask_x & (~mask_y) & (~mask_z) & tmp[y_col].ne(0)
y_fix = mask_y & (~mask_x) & (~mask_z) & tmp[x_col].ne(0)
z_fix = mask_z & (~mask_x) & (~mask_y)

df.loc[x_fix, x_col] = (tmp.loc[x_fix, z_col] / tmp.loc[x_fix, y_col]).round(2)
df.loc[y_fix, y_col] = (tmp.loc[y_fix, z_col] / tmp.loc[y_fix, x_col]).round(2)
df.loc[z_fix, z_col] = (tmp.loc[z_fix, x_col] * tmp.loc[z_fix, y_col]).round(2)

# Substituir -1 restantes por 0
for col in [x_col, y_col, z_col]:
    df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)
    df.loc[df[col].eq(-1), col] = 0

print(f"{x_col} corrigidos:", int(x_fix.sum()))
print(f"{y_col} corrigidos:", int(y_fix.sum()))
print(f"{z_col} corrigidos:", int(z_fix.sum()))
print("Substituições por 0 concluídas quando duas colunas estavam zeradas ou restavam -1.")



claims_count corrigidos: 3298
avg_claim_amount corrigidos: 3246
total_claims_paid corrigidos: 3224
Substituições por 0 concluídas quando duas colunas estavam zeradas ou restavam -1.


5.2 colunas ano e mês premium

In [6]:
import numpy as np
import pandas as pd

a_col = "annual_premium"
b_col = "monthly_premium"

tmp = df[[a_col, b_col]].apply(pd.to_numeric, errors="coerce")

mask_a = df[a_col].astype(str).eq("-1") | tmp[a_col].eq(-1)
mask_b = df[b_col].astype(str).eq("-1") | tmp[b_col].eq(-1)

# Caso ambas sejam -1 → definir 0
both_neg1 = mask_a & mask_b
df.loc[both_neg1, [a_col, b_col]] = 0

# Atualiza máscaras após zerar ambas
tmp = df[[a_col, b_col]].apply(pd.to_numeric, errors="coerce")
mask_a = df[a_col].astype(str).eq("-1") | tmp[a_col].eq(-1)
mask_b = df[b_col].astype(str).eq("-1") | tmp[b_col].eq(-1)

# Calcular a partir de b: a = 12 * b
a_fix = mask_a & (~mask_b)
df.loc[a_fix, a_col] = (tmp.loc[a_fix, b_col] * 12).round(2)

# Calcular a partir de a: b = a / 12
b_fix = mask_b & (~mask_a)
df.loc[b_fix, b_col] = (tmp.loc[b_fix, a_col] / 12).round(2)

# Substituir -1 restantes por 0
for col in [a_col, b_col]:
    df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)
    df.loc[df[col].eq(-1), col] = 0

print(f"{a_col} corrigidos:", int(a_fix.sum()))
print(f"{b_col} corrigidos:", int(b_fix.sum()))
print("Substituições finais por 0 concluídas quando ambas eram -1.")


annual_premium corrigidos: 5776
monthly_premium corrigidos: 5808
Substituições finais por 0 concluídas quando ambas eram -1.


5.3 outliers remove

In [7]:
# 5.3 - Augusto editou aqui
col_cost = "annual_medical_cost"

# Converter para numérico sem alterar df original
cost_num = pd.to_numeric(df[col_cost], errors="coerce")

# Considerar apenas valores >= 0 para o cálculo dos limites
valid_mask = cost_num >= 0
valid_costs = cost_num[valid_mask]

# Calcular IQR
q1 = valid_costs.quantile(0.25)
q3 = valid_costs.quantile(0.75)  # VOLTAMOS O CERTO: Q3 = 0.75
iqr = q3 - q1

lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr

print(f"Q1: {q1:.2f}, Q3: {q3:.2f}, IQR: {iqr:.2f}")
print(f"Limites para não-outliers: [{lower:.2f}, {upper:.2f}]")

# Identificar valores outliers
outlier_mask = valid_mask & ~cost_num.between(lower, upper)

# Quantidade de outliers
removed = int(outlier_mask.sum())
print(f"Valores outliers identificados em {col_cost}: {removed}")

# Substituir APENAS os valores outliers por NaN (não remove linhas)
df.loc[outlier_mask, col_cost] = np.nan

print("Outliers removidos (definidos como NaN).")
print("Dimensões permanecem as mesmas:", df.shape)


Q1: 1174.03, Q3: 3701.29, IQR: 2527.26
Limites para não-outliers: [-2616.85, 7492.18]
Valores outliers identificados em annual_medical_cost: 3921
Outliers removidos (definidos como NaN).
Dimensões permanecem as mesmas: (64800, 11)


6- exportação

In [8]:
df.to_csv(OUTPUT_FILE, index=False)
print(f"Arquivo salvo em: {OUTPUT_FILE}")


Arquivo salvo em: /Users/augusto/Library/Mobile Documents/com~apple~CloudDocs/git/avalensurance-bia/data/costs_cleaned.csv


7- pegando onde tem omissão

In [9]:
from pathlib import Path
import pandas as pd

ROOT = Path.cwd().parents[1]
DATA_DIR = ROOT / "data"
OUTPUT_FALTANTES = DATA_DIR / "faltantes_cost.csv"

mask_minus1 = df.replace("-1", -1).eq(-1)
counts_minus1 = mask_minus1.sum()
print("Contagem de -1 por coluna (após tratamentos x,y,z):")
print(counts_minus1[counts_minus1 > 0])

faltantes = df[mask_minus1.any(axis=1)].copy()
faltantes.to_csv(OUTPUT_FALTANTES, index=False)
print(f"Arquivo 'faltantes_cost.csv' criado com {faltantes.shape[0]} linhas contendo -1 após os tratamentos.")


Contagem de -1 por coluna (após tratamentos x,y,z):
annual_medical_cost    6380
dtype: int64
Arquivo 'faltantes_cost.csv' criado com 6380 linhas contendo -1 após os tratamentos.


/var/folders/99/0hmnt5l13rn3b4mhfw326ws00000gn/T/ipykernel_22777/371089213.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mask_minus1 = df.replace("-1", -1).eq(-1)
